# Data Load


In [1]:
import numpy as np
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
#%matplotlib tk

#import matplotlib 
#matplotlib.font_manager._rebuild()

In [3]:
data=pd.read_csv("C:/Users/sohee/Desktop/KW/산학연계(졸작)/산학졸작_openUP_Data/kwproja_data_.csv",encoding='utf-8')

# EDA 

153395 rows X 31 columns

- 매장 속성 정보
  - shop_code -> 식별자 feature, input feature로는 사용하지 않지만 분류를 위해서는 사용할 수 있을 것 같음  
  - shop_name -> input feature로는 사용하지 않음 (NLP deep learning 가능성 있음)
  - longtitude : 경도, latitude : 위도 -> 매장 위치 (회사 근처, 학교 근처 등 매출 영향성 있음) -> 군집화, labeling 필요
  - shop_type_big -> 13 category  -> 업종 (매출 영향성 있음)
  - shop_type_small -> 367 category

- 매출 정보
  - date -> 12 category, 201702~ 201801 까지의 data
  - monthly_gain / avearge_sale_price = 한달 총 판매수

- 매출 통계 정보
  - weekday0~6 : 일 ~ 월
  - time -> 05_10 / 10_14 / 14_18 / 18_20 / 20_22 / 22_24 / 24_05 => 시간대가 동일하지 않음
    - 새벽대는 찾는 손님 적어 시간 길게 잡았을 것
    - 저녁 시간대는 가장 매출이 많이 이뤄지는 prime time 이라 시간대를 짧게 잡았을 것
  - female/male : 20/30/40/50 -> 8 columns

##### monthly_gain과 average_sale_price 중 어느 것을 y값으로 둘 것인가? 
- 월매출 예측 문제로 가정하고 montly_gain 을 y값으로 예측하는 모델 만들기

##### shop_code는 input feature에 넣어야 하는가?
- 특별한 브랜드가 y값을 결정하는 과적합 요소가 될 수 있으므로
- X 에서 shop code, shop name 제외하는 것도 방법
- 어느 위치에 어떤 업종으로 어떤 객단가인 매장을 오픈하면 월매출이 어떻게 될까? 문제
 - X: shop type big, shop type small, longitude, latitude, avg_sale_price, 
 - y: montly_gain

In [4]:
data.columns

Index(['date', 'shop_code', 'shop_name', 'shop_type_big', 'shop_type_small',
       'longitude', 'latitude', 'monthly_gain', 'average_sale_price',
       'female_20', 'female_30', 'female_40', 'female_50', 'male_20',
       'male_30', 'male_40', 'male_50', 'weekday0', 'weekday1', 'weekday2',
       'weekday3', 'weekday4', 'weekday5', 'weekday6', 'time_05_10',
       'time_10_14', 'time_14_18', 'time_18_20', 'time_20_22', 'time_22_24',
       'time_00_05'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153395 entries, 0 to 153394
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   date                153395 non-null  int64  
 1   shop_code           153395 non-null  int64  
 2   shop_name           153395 non-null  object 
 3   shop_type_big       148927 non-null  object 
 4   shop_type_small     148927 non-null  object 
 5   longitude           153395 non-null  float64
 6   latitude            153395 non-null  float64
 7   monthly_gain        153395 non-null  float64
 8   average_sale_price  153395 non-null  int64  
 9   female_20           153395 non-null  float64
 10  female_30           153395 non-null  float64
 11  female_40           153395 non-null  float64
 12  female_50           153395 non-null  float64
 13  male_20             153395 non-null  float64
 14  male_30             153395 non-null  float64
 15  male_40             153395 non-nul

# Preprocessing
- missing value 제거 
    - monthly_gain : 3,605 제거 -> 149,790
    - shop_type_big : 4,303 제거 -> 145,487 
    - gender feature : 1,187 -> 144,300
- shop_code, shop_name : 식별자 feature 이므로 drop 
- date : 아직은 쓸 수 없으므로 drop
    - 여기까지 총 144,300 X 27
- shop_type_big(13), shop_type_small(367) : label encodding
- longitude, latitude : clustering을 통해 labeling 후 해당 두 열은 drop 
- MinMaxSaclar 정규화 -> 정규화 column의 범위는??

In [6]:
# 원본 data와 따로 관리 -> original data = data, input data = input_data 
# feature drop
input_data = data.copy()
input_data = input_data.drop(['date', 'shop_code', 'shop_name'], axis=1)
#input_data

In [7]:
# missing value drop - monthly_gain
null_index = input_data[input_data['monthly_gain']==0].index 
#print(len(drop_index)) -> 3605

input_data = input_data.drop(null_index)
#input_data

In [8]:
# missing value drop - shop_type_big
null_index = input_data[input_data['shop_type_big'].isnull()==True].index
input_data = input_data.drop(null_index)
#input_data

In [9]:
# There is no "null" in "shop_type_small" feature 

#print(df_check['shop_type_small'].unique())
print(len(input_data['shop_type_small'].unique()))
print(input_data['shop_type_small'].isnull().value_counts())

367
False    145487
Name: shop_type_small, dtype: int64


In [10]:
input_data[input_data[['female_20', 'female_30', 'female_40', 'female_50', 'male_20',
       'male_30', 'male_40', 'male_50']].sum(axis=1)==0]

,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,female_20,female_30,female_40,female_50,...,weekday4,weekday5,weekday6,time_05_10,time_10_14,time_14_18,time_18_20,time_20_22,time_22_24,time_00_05
221,음식,기타일반 유흥주점,127.034695,37.527644,1257500.0,250000,0.0,0.0,0.0,0.0,...,100.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,100.00
287,도매/유통/무역,철강판재 도매,126.904171,37.515872,622715.5,123310,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,100.0,0.0,0.0,0.00,0.00
296,소매,선물용품 판매,126.930103,37.555125,603600.0,120000,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,100.0,0.0,0.0,0.00,0.00
538,소매,신발 판매,126.984743,37.563528,523120.0,104000,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,100.0,0.0,0.0,0.00,0.00
748,소매,슈퍼마켓 체인점,126.923453,37.556142,4471670.0,80818,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,78.63,21.37,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153239,소매,꽃집/꽃배달,127.024831,37.524013,1802500.0,350000,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,100.0,0.0,0.0,0.00,0.00
153259,소매,가전제품 소매,127.098023,37.511472,2571910.0,499400,0.0,0.0,0.0,0.0,...,100.00,0.00,0.0,0.00,100.00,0.0,0.0,0.0,0.00,0.00
153293,소매,꽃집/꽃배달,127.024612,37.520046,1609650.0,315000,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,100.00,0.0,0.0,0.0,0.00,0.00
153301,의료,병원-성형외과,127.027963,37.523336,45114000.0,8760000,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,100.00,0.0,0.0,0.0,0.00,0.00


In [11]:
# missing value drop - gender feature 
# So we finally get 144,300 x 28 data 

null_index = input_data[input_data[['female_20', 'female_30', 'female_40', 'female_50', 'male_20',
       'male_30', 'male_40', 'male_50']].sum(axis=1)==0].index
input_data = input_data.drop(null_index)
input_data

,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,female_20,female_30,female_40,female_50,...,weekday4,weekday5,weekday6,time_05_10,time_10_14,time_14_18,time_18_20,time_20_22,time_22_24,time_00_05
0,음식,카페 일반,126.927911,37.558043,5.696400e+06,9982,9.58,5.43,18.68,4.63,...,17.15,11.92,6.34,1.15,72.78,23.14,2.93,0.00,0.00,0.00
1,음식,한정식/백반 전문점,127.022532,37.506856,8.496120e+06,14503,5.08,12.12,9.68,8.15,...,19.38,16.35,10.82,0.83,76.82,13.25,9.09,0.00,0.00,0.00
2,음식,고기구이 전문점,127.025013,37.505696,1.036515e+08,72527,10.51,9.64,4.05,3.57,...,21.18,21.59,18.04,4.64,8.50,2.40,18.67,50.05,9.84,5.89
3,음식,고기구이 전문점,127.025013,37.505696,9.678575e+07,70225,12.10,11.48,6.31,4.77,...,26.08,22.60,15.51,2.68,8.09,5.77,10.15,54.61,16.25,2.45
4,소매,스포츠용품 판매,126.903306,37.517063,1.266359e+08,91884,15.04,10.48,16.90,9.56,...,16.44,10.38,22.71,0.43,26.38,39.20,25.30,8.69,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153390,학문/교육,학원-기타,127.040033,37.525081,2.277527e+07,1114250,0.00,0.00,25.35,22.82,...,26.48,0.00,0.00,73.52,26.48,0.00,0.00,0.00,0.00,0.00
153391,소매,화장품 판매,127.046604,37.524470,1.555300e+06,33556,0.00,0.00,46.48,44.72,...,5.30,23.18,20.53,23.18,52.65,13.58,10.60,0.00,0.00,0.00
153392,소매,장난감/오락기기/용품 판매,127.060835,37.509203,3.134106e+08,39543,31.27,14.62,9.13,2.91,...,13.02,13.71,13.84,6.60,44.94,36.45,12.01,0.00,0.00,0.00
153393,의료,약국,127.103491,37.513101,7.927725e+07,9874,23.15,18.09,9.60,9.93,...,10.42,11.71,16.84,4.45,25.70,30.00,22.70,16.61,0.55,0.00


In [12]:
# issue -> shop_type_big에 관한 2번째 issue를 참고하세요!
# issue -> '음식'과 '음식 ' 두 개로 count 되고 있습니다. 아래 코드로 수정하시길 바랍니다

input_data.loc[data['shop_type_big'] == '음식 ', 'shop_type_big'] = '음식'
print(input_data['shop_type_big'].nunique())
print(input_data['shop_type_big'].unique())

13
['음식' '소매' '의료' '숙박' '관광/여가/오락' '학문/교육' '생활서비스' '스포츠' '도매/유통/무역' '제조'
 '부동산' '전자/정보통신' '기술서비스']


In [13]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

input_data['shop_type_big'] = le.fit_transform(list(input_data['shop_type_big']))   # fit transform으로 한번에 처리 가능
le.classes_

array(['관광/여가/오락', '기술서비스', '도매/유통/무역', '부동산', '생활서비스', '소매', '숙박', '스포츠',
       '음식', '의료', '전자/정보통신', '제조', '학문/교육'], dtype='<U8')

In [14]:
le = LabelEncoder()

input_data['shop_type_small'] = le.fit_transform(list(input_data['shop_type_small'])) 
#le.classes_

In [15]:
# Labeling - KMeans Clustering 
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=9).fit(input_data[['latitude', 'longitude']])
kmeans.cluster_centers_
kmeans.labels_

array([1, 0, 0, ..., 8, 6, 6])

In [16]:
# longitude + latitude = geo 
# So we get inpute_data = 144,300 X 27
input_data['geo'] = kmeans.labels_
input_data = input_data.drop(['longitude', 'latitude'], axis=1)
input_data

,shop_type_big,shop_type_small,monthly_gain,average_sale_price,female_20,female_30,female_40,female_50,male_20,male_30,...,weekday5,weekday6,time_05_10,time_10_14,time_14_18,time_18_20,time_20_22,time_22_24,time_00_05,geo
0,8,304,5.696400e+06,9982,9.58,5.43,18.68,4.63,27.62,8.89,...,11.92,6.34,1.15,72.78,23.14,2.93,0.00,0.00,0.00,1
1,8,346,8.496120e+06,14503,5.08,12.12,9.68,8.15,9.68,9.75,...,16.35,10.82,0.83,76.82,13.25,9.09,0.00,0.00,0.00,0
2,8,17,1.036515e+08,72527,10.51,9.64,4.05,3.57,23.30,31.56,...,21.59,18.04,4.64,8.50,2.40,18.67,50.05,9.84,5.89,0
3,8,17,9.678575e+07,70225,12.10,11.48,6.31,4.77,15.64,30.60,...,22.60,15.51,2.68,8.09,5.77,10.15,54.61,16.25,2.45,0
4,5,170,1.266359e+08,91884,15.04,10.48,16.90,9.56,21.64,11.67,...,10.38,22.71,0.43,26.38,39.20,25.30,8.69,0.00,0.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153390,12,323,2.277527e+07,1114250,0.00,0.00,25.35,22.82,0.00,26.48,...,0.00,0.00,73.52,26.48,0.00,0.00,0.00,0.00,0.00,3
153391,5,357,1.555300e+06,33556,0.00,0.00,46.48,44.72,0.00,0.00,...,23.18,20.53,23.18,52.65,13.58,10.60,0.00,0.00,0.00,3
153392,5,257,3.134106e+08,39543,31.27,14.62,9.13,2.91,19.56,12.47,...,13.71,13.84,6.60,44.94,36.45,12.01,0.00,0.00,0.00,8
153393,9,190,7.927725e+07,9874,23.15,18.09,9.60,9.93,11.81,10.03,...,11.71,16.84,4.45,25.70,30.00,22.70,16.61,0.55,0.00,6


In [24]:
# Normalization
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

scaler = StandardScaler()

#scale_cols = input_data.columns.tolist()
scaled_data = scaler.fit_transform(input_data['average_sale_price'])
scaled_data = pd.DataFrame(scaled_data)
scaled_data.columns = scale_cols 

input_data['average_sale_price'] = scaled_data
input_data

ValueError: Expected 2D array, got 1D array instead:
array=[  9982.  14503.  72527. ...  39543.   9874. 293140.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
ax = plt.subplots()
ax = sns.distplot(input_data['monthly_gain'], hist=False)
ax.set_title('Total Gain Density')
ax.set_xlabel('Monthly Gain')
ax.set_ylabel('Unit Probability')
print(input_data['monthly_gain'].value_counts())

# Data Split

In [18]:
input_data_y = input_data['monthly_gain'].copy()
input_data_X = input_data.drop(['monthly_gain'], axis=1)

In [19]:
from sklearn.model_selection import train_test_split

tr_val_X, test_X, tr_val_y, test_y = train_test_split(
    input_data_X, 
    input_data_y, 
    test_size = 0.2,      
    shuffle=True,         
    random_state=42)      
print(len(test_X))

train_X, valid_X, train_y, valid_y = train_test_split(
    tr_val_X, 
    tr_val_y, 
    test_size = 0.2,      
    shuffle=True,         
    random_state=42) 

print(len(train_X))
print(len(valid_X))

28860
92352
23088


# Modeling
- XGB
- LGBM

In [20]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold, TimeSeriesSplit,GridSearchCV
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor
from lightgbm import plot_importance 
from xgboost import XGBRegressor
from xgboost import plot_importance

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [21]:
model_xgb = XGBRegressor()

model_xgb.fit(train_X, train_y, early_stopping_rounds=100, eval_set=[(valid_X, valid_y)],verbose=True)
pred_xgb = model_xgb.predict(test_X)

[0]	validation_0-rmse:708255040.00000
[1]	validation_0-rmse:614250944.00000
[2]	validation_0-rmse:552968320.00000
[3]	validation_0-rmse:533570464.00000
[4]	validation_0-rmse:535917952.00000
[5]	validation_0-rmse:514039616.00000
[6]	validation_0-rmse:509163744.00000
[7]	validation_0-rmse:501134656.00000
[8]	validation_0-rmse:494723200.00000
[9]	validation_0-rmse:477173440.00000
[10]	validation_0-rmse:475760320.00000
[11]	validation_0-rmse:466142976.00000
[12]	validation_0-rmse:464330528.00000
[13]	validation_0-rmse:456073696.00000
[14]	validation_0-rmse:456894784.00000
[15]	validation_0-rmse:456319680.00000
[16]	validation_0-rmse:451715680.00000
[17]	validation_0-rmse:450890848.00000
[18]	validation_0-rmse:447095520.00000
[19]	validation_0-rmse:446710752.00000
[20]	validation_0-rmse:442932192.00000
[21]	validation_0-rmse:442989696.00000
[22]	validation_0-rmse:435869920.00000
[23]	validation_0-rmse:435967488.00000
[24]	validation_0-rmse:433792288.00000
[25]	validation_0-rmse:433328000.00

In [22]:
model_lgbm = LGBMRegressor()

model_lgbm.fit(train_X, train_y, early_stopping_rounds=100, eval_set=[(valid_X, valid_y)], verbose=True)
pred_lgbm = model_lgbm.predict(test_X)

c:\users\sohee\appdata\local\programs\python\python38\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sohee\appdata\local\programs\python\python38\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 6.64693e+17
[2]	valid_0's l2: 5.9833e+17
[3]	valid_0's l2: 5.42432e+17
[4]	valid_0's l2: 4.86401e+17
[5]	valid_0's l2: 4.40924e+17
[6]	valid_0's l2: 3.99982e+17
[7]	valid_0's l2: 3.69661e+17
[8]	valid_0's l2: 3.35021e+17
[9]	valid_0's l2: 3.11705e+17
[10]	valid_0's l2: 2.9092e+17
[11]	valid_0's l2: 2.75426e+17
[12]	valid_0's l2: 2.6219e+17
[13]	valid_0's l2: 2.45254e+17
[14]	valid_0's l2: 2.38984e+17
[15]	valid_0's l2: 2.28045e+17
[16]	valid_0's l2: 2.23138e+17
[17]	valid_0's l2: 2.12357e+17
[18]	valid_0's l2: 2.03902e+17
[19]	valid_0's l2: 1.96919e+17
[20]	valid_0's l2: 1.87703e+17
[21]	valid_0's l2: 1.82325e+17
[22]	valid_0's l2: 1.76222e+17
[23]	valid_0's l2: 1.72549e+17
[24]	valid_0's l2: 1.68235e+17
[25]	valid_0's l2: 1.64482e+17
[26]	valid_0's l2: 1.58058e+17
[27]	valid_0's l2: 1.53936e+17
[28]	valid_0's l2: 1.52115e+17
[29]	valid_0's l2: 1.48393e+17
[30]	valid_0's l2: 1.42671e+17
[31]	valid_0's l2: 1.40835e+17
[32]	valid_0's l2: 1.38991e+17
[33]	valid_0's l2: 1

In [23]:
def get_rmse_pred(preds):
    for key in preds.keys():
        pred_value=preds[key]
        mse=mean_squared_error(test_y,pred_value)
        rmse=np.sqrt(mse)
        print('{0} 모델의 MSE: {1}'.format(key,mse))
        print('{0} 모델의 RMSE: {1}'.format(key,rmse))
        
pred=0.5*pred_xgb+0.5*pred_lgbm
preds={"혼합" : pred,
      'XGBM' : pred_xgb,
      'LGBM' : pred_lgbm}

get_rmse_pred(preds)

혼합 모델의 MSE: 1.1753860484308622e+17
혼합 모델의 RMSE: 342839036.3466305
XGBM 모델의 MSE: 1.273560595653042e+17
XGBM 모델의 RMSE: 356869807.58436847
LGBM 모델의 MSE: 1.259462502231339e+17
LGBM 모델의 RMSE: 354889067.48889
